# How to use the invenio API

- run API and UI with:

  ```bash
  ./run.sh
  ```

## Useful links

- [Invenio API requests](https://invenio.readthedocs.io/en/latest/getting-started/quickstart/crud-operations.html)
- [Invenio REST API](https://inveniordm.docs.cern.ch/reference/rest_api_index/) (they are lying about authentication only with Access Tokens, you can authenticate as shown below)
- [NRP Docs](https://nrp-cz.github.io/docs)
- [NRP models](https://narodni-repozitar.github.io/developer-docs/docs/technology/invenio/nrp-toolchain/edit-metadata)

## Actions

In [4]:
import requests
import json

API_ENDPOINT = 'https://mdrepo.eu/api'
# API_ENDPOINT = 'https://inveniordm.web.cern.ch/api'

# test if API is up
response = requests.get(f'{API_ENDPOINT}/login')
if response.status_code != 405:
    raise Exception('API returned with code', response.status_code)

### Login

In [7]:
data = {
    'email': 'test@test.com',
    'password': '123456'
}

response = requests.post(
    f'{API_ENDPOINT}/login',
    data=data
)
print(response.json())

session_cookie = response.cookies['session']
print(response.cookies)
print('Session cookie:', session_cookie)

{'confirmed_at': '2024-11-16T11:43:21.328322', 'email': 'test@test.com', 'id': 1, 'last_login_at': '2024-11-19T12:10:09.241273', 'roles': []}
<RequestsCookieJar[<Cookie session=99c57be85d61c307_673c8082.lJ8X2Hg9m1y52uekKcxc_UL1uCA for mdrepo.eu/>]>
Session cookie: 99c57be85d61c307_673c8082.lJ8X2Hg9m1y52uekKcxc_UL1uCA


### Search

In [6]:
params = {
    'q': 'LRIPCCPVNLKRLLVVVVVVVLVVVVIVGALLMGL',
    'size': 10,
    'page': 1
}
response = requests.get(f'{API_ENDPOINT}/experiments', params=params)
print(response.json())

{'hits': {'hits': [{'id': '3kkb0-7ka94', 'created': '2024-11-16T13:19:41.469207+00:00', 'updated': '2024-11-16T13:19:41.614718+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/requests/applicable', 'communities': {'e9937d36-7ceb-48bf-891d-f3ed73c296fb': {'self': 'https://mdrepo.eu/api/communities/e9937d36-7ceb-48bf-891d-f3ed73c296fb', 'self_html': 'https://mdrepo.eu/communities/ceitec/records'}}, 'draft': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/draft', 'files': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/files', 'latest': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/3kkb0-7ka94/latest', 'publish': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94', 'requests': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94/requests', 'self': 'https://mdrepo.eu/api/experiments/3kkb0-7ka94', 'self_html': 'https://m

### Create record

In [14]:
# XXX: session_cookie is required to create a record
cookies = {'session': session_cookie}

with open('demo-data/example_metadata.json', 'r') as f:
    metadata = json.load(f)

data = {
    'files': {'enabled': True},
    'parent' : {
        'communities': {'default': 'ceitec'}
    },
    'metadata': metadata,
}

response = requests.post(
    f'{API_ENDPOINT}/experiments',
    cookies=cookies,
    json=data
)

print('Status:', response.status_code)
print('Full response:', response.text)

if 'errors' in response.json():
    print('Errors:', response.json()['errors'])

if response.json().get("message", "") == "The persistent identifier does not exist.":
    print(f"⚠️Check if the group '{data['parent']['communities']['default']}' exists")

record_id = response.json().get('id', None)
print('Record ID:', record_id)

Status: 201
Full response: {"id": "emqpz-mx665", "created": "2024-11-19T12:38:13.233997+00:00", "updated": "2024-11-19T12:38:13.354460+00:00", "links": {"applicable-requests": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/requests/applicable", "communities": {"e9937d36-7ceb-48bf-891d-f3ed73c296fb": {"self": "https://mdrepo.eu/api/communities/e9937d36-7ceb-48bf-891d-f3ed73c296fb", "self_html": "https://mdrepo.eu/communities/ceitec/records"}}, "draft": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft", "edit_html": "https://mdrepo.eu/experiments/emqpz-mx665/edit", "files": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files", "latest": "https://mdrepo.eu/api/experiments/emqpz-mx665/versions/latest", "latest_html": "https://mdrepo.eu/experiments/emqpz-mx665/latest", "publish": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/actions/publish", "record": "https://mdrepo.eu/api/experiments/emqpz-mx665", "requests": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/re

### Edit record

In [33]:
cookies = {'session': session_cookie}

# random change to the metadata
metadata['simulations'][0]['file_identification']['description'] = "I just edited this record :P"

data = {
    'metadata': metadata,
}

response = requests.put(
    f'{API_ENDPOINT}/experiments/{record_id}/draft',
    cookies=cookies,
    json=data
)

print('Status:', response.status_code)
print('Full response:', response.text)

if 'errors' in response.json():
    print('Errors:', response.json()['errors'])

Status: 200
Full response: {"id": "va12d-wrk79", "created": "2024-11-16T14:49:28.526990+00:00", "updated": "2024-11-16T14:54:36.775259+00:00", "links": {"applicable-requests": "https://mdrepo.eu/api/experiments/va12d-wrk79/draft/requests/applicable", "communities": {"e9937d36-7ceb-48bf-891d-f3ed73c296fb": {"self": "https://mdrepo.eu/api/communities/e9937d36-7ceb-48bf-891d-f3ed73c296fb", "self_html": "https://mdrepo.eu/communities/ceitec/records"}}, "draft": "https://mdrepo.eu/api/experiments/va12d-wrk79/draft", "edit_html": "https://mdrepo.eu/experiments/va12d-wrk79/edit", "files": "https://mdrepo.eu/api/experiments/va12d-wrk79/draft/files", "latest": "https://mdrepo.eu/api/experiments/va12d-wrk79/versions/latest", "latest_html": "https://mdrepo.eu/experiments/va12d-wrk79/latest", "publish": "https://mdrepo.eu/api/experiments/va12d-wrk79/draft/actions/publish", "record": "https://mdrepo.eu/api/experiments/va12d-wrk79", "requests": "https://mdrepo.eu/api/experiments/va12d-wrk79/draft/re

### View record

In [29]:
# in UI
# draft:        /experiments/<record_id>/preview
# published:    /experiments/<record_id>

cookies = {'session': session_cookie}

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/draft',
    cookies=cookies
)
print(response.json())

{'id': 'va12d-wrk79', 'created': '2024-11-16T14:49:28.526990+00:00', 'updated': '2024-11-16T14:49:59.757981+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/va12d-wrk79/draft/requests/applicable', 'communities': {'e9937d36-7ceb-48bf-891d-f3ed73c296fb': {'self': 'https://mdrepo.eu/api/communities/e9937d36-7ceb-48bf-891d-f3ed73c296fb', 'self_html': 'https://mdrepo.eu/communities/ceitec/records'}}, 'draft': 'https://mdrepo.eu/api/experiments/va12d-wrk79/draft', 'edit_html': 'https://mdrepo.eu/experiments/va12d-wrk79/edit', 'files': 'https://mdrepo.eu/api/experiments/va12d-wrk79/draft/files', 'latest': 'https://mdrepo.eu/api/experiments/va12d-wrk79/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/va12d-wrk79/latest', 'publish': 'https://mdrepo.eu/api/experiments/va12d-wrk79/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/va12d-wrk79', 'requests': 'https://mdrepo.eu/api/experiments/va12d-wrk79/draft/requests', 'self': 'https://m

### List files of a record

In [11]:
cookies = {'session': session_cookie}

response = requests.get(f'{API_ENDPOINT}/experiments/{record_id}/draft/files', cookies=cookies)
print(response.json())

{'enabled': True, 'links': {'self': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft/files'}, 'entries': [{'key': 'SPC.tpr', 'storage_class': 'L', 'checksum': 'md5:eb43ff185b5e6bdc3a2d0faea2e1db19', 'size': 5303276, 'created': '2024-11-19T12:15:33.729098+00:00', 'updated': '2024-11-19T12:15:34.616217+00:00', 'status': 'completed', 'mimetype': 'application/octet-stream', 'version_id': '13f42090-6d86-4e77-94ec-9e9d9ad1d706', 'file_id': '7594b7a7-833d-4988-bd07-2b03f6eb6331', 'bucket_id': '89d4b402-52f5-463f-9650-39d4a0dcae06', 'metadata': None, 'access': {'hidden': False}, 'links': {'commit': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft/files/SPC.tpr/commit', 'content': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft/files/SPC.tpr/content', 'preview': 'https://mdrepo.eu/experiments/n5ajx-add49/files/SPC.tpr/preview', 'self': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft/files/SPC.tpr'}}], 'default_preview': None, 'order': []}


### Upload file to a record

In [15]:
local_file_name = 'demo-data/SPC.tpr'
remote_file_name = 'SPC.tpr'

cookies = {'session': session_cookie}

# set file metadata
data = [{'key': remote_file_name}]
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files',
    cookies=cookies,
    json=data
)
print('POST metadata:', response.text)

# upload file
with open(local_file_name, 'rb') as f:
    response = requests.put(
        f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/content',
        cookies=cookies,
        data=f,
        stream=True
    )
    print('PUT file:', response.text)

# commit file
response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/files/{remote_file_name}/commit',
    cookies=cookies
)
print('POST commit:', response.text)


POST metadata: {"enabled": true, "links": {"self": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files"}, "entries": [{"key": "SPC.tpr", "created": "2024-11-19T12:38:20.295378+00:00", "updated": "2024-11-19T12:38:20.301208+00:00", "status": "pending", "metadata": null, "access": {"hidden": false}, "links": {"commit": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files/SPC.tpr/commit", "content": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files/SPC.tpr/content", "preview": "https://mdrepo.eu/experiments/emqpz-mx665/files/SPC.tpr/preview", "self": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files/SPC.tpr"}}], "default_preview": null, "order": []}
PUT file: {"key": "SPC.tpr", "created": "2024-11-19T12:38:20.295378+00:00", "updated": "2024-11-19T12:38:20.301208+00:00", "status": "pending", "metadata": null, "access": {"hidden": false}, "links": {"commit": "https://mdrepo.eu/api/experiments/emqpz-mx665/draft/files/SPC.tpr/commit", "content": "https://mdrepo.

### Download file from a record

In [17]:
# XXX: record must be published (/draft only sends S3 direct url)

cookies = {'session': session_cookie}
remote_file_name = 'SPC.tpr'

response = requests.get(
    f'{API_ENDPOINT}/experiments/{record_id}/files/{remote_file_name}/content',
    cookies=cookies
)
if response.status_code >= 400:
    print(response.text)
    response.raise_for_status()

with open(remote_file_name, 'wb') as f:
    f.write(response.content)

{"status": 404, "message": "Not found."}


HTTPError: 404 Client Error: NOT FOUND for url: https://mdrepo.eu/api/experiments/emqpz-mx665/files/SPC.tpr/content

### Publish record

In [12]:
cookies = {'session': session_cookie}

response = requests.post(
    f'{API_ENDPOINT}/experiments/{record_id}/draft/actions/publish',
    cookies=cookies
)
print('POST publish:', response.json())

POST publish: {'id': 'n5ajx-add49', 'created': '2024-11-19T12:16:11.013558+00:00', 'updated': '2024-11-19T12:16:11.151842+00:00', 'links': {'applicable-requests': 'https://mdrepo.eu/api/experiments/n5ajx-add49/requests/applicable', 'communities': {'e9937d36-7ceb-48bf-891d-f3ed73c296fb': {'self': 'https://mdrepo.eu/api/communities/e9937d36-7ceb-48bf-891d-f3ed73c296fb', 'self_html': 'https://mdrepo.eu/communities/ceitec/records'}}, 'draft': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft', 'files': 'https://mdrepo.eu/api/experiments/n5ajx-add49/files', 'latest': 'https://mdrepo.eu/api/experiments/n5ajx-add49/versions/latest', 'latest_html': 'https://mdrepo.eu/experiments/n5ajx-add49/latest', 'publish': 'https://mdrepo.eu/api/experiments/n5ajx-add49/draft/actions/publish', 'record': 'https://mdrepo.eu/api/experiments/n5ajx-add49', 'requests': 'https://mdrepo.eu/api/experiments/n5ajx-add49/requests', 'self': 'https://mdrepo.eu/api/experiments/n5ajx-add49', 'self_html': 'https://mdrepo